# SQL-ПРОЕКТ

## Цели исследования

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
! pip install sqlalchemy

In [7]:
import pandas as pd
from sqlalchemy import text, create_engine

In [8]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [9]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

ModuleNotFoundError: No module named 'psycopg2'

Создадим функцию, которая значительно облегчит жизнь при выполнении запросов

In [10]:
con=engine.connect()

NameError: name 'engine' is not defined

In [11]:
def wanna_sql(i):
    return pd.io.sql.read_sql(sql=text(i), con=con)

## Исследуем таблицы — выведем первые строки

Для начала посмотрим таблицы, с которыми предстоит работать

In [12]:
df_books = '''SELECT * FROM books LIMIT 5'''
wanna_sql(df_books)

NameError: name 'con' is not defined

In [ ]:
df_authors = '''SELECT * FROM authors LIMIT 5'''
wanna_sql(df_authors)

In [ ]:
df_ratings = '''SELECT * FROM ratings LIMIT 5'''
wanna_sql(df_ratings)

In [ ]:
df_reviews = '''SELECT * FROM reviews LIMIT 5'''
wanna_sql(df_reviews)

In [ ]:
df_publishers = '''SELECT * FROM publishers LIMIT 5'''
wanna_sql(df_publishers)

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [ ]:
exersice_1 = '''
SELECT COUNT(DISTINCT book_id) 
FROM books
WHERE publication_date >= '2000-01-01'
'''
wanna_sql(exersice_1)

В итоге получилось, что 821 книга вышла после 1 января 2000 года

## Для каждой книги посчитаем кол-во обзоров и среднюю оценку

In [ ]:
ex = '''
SELECT COUNT(book_id)
       
FROM books 
'''
wanna_sql(ex)

In [ ]:
exersice_2 = '''
WITH mean_rat AS (
SELECT book_id,
       AVG(rating) AS mean_rating
FROM ratings 
GROUP BY book_id),

n_rev AS (
SELECT book_id,
       COUNT(review_id) AS n_reviews
FROM reviews  
GROUP BY book_id),

n_titles AS (
SELECT book_id,
       title
FROM books
GROUP BY book_id)

SELECT nt.title AS titel,
       mn.mean_rating,
       nr.n_reviews
FROM n_titles AS nt
LEFT JOIN mean_rat AS mn ON nt.book_id=mn.book_id
LEFT JOIN n_rev AS nr ON nr.book_id=nt.book_id
GROUP BY
titel,
mn.mean_rating,
nr.n_reviews
ORDER BY
mn.mean_rating DESC,
nr.n_reviews DESC
LIMIT 3
'''
wanna_sql(exersice_2)

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [ ]:
exersice_3 = '''
SELECT
      p.publisher AS titel,
      COUNT(DISTINCT b.book_id) AS n_books
FROM publishers AS p
LEFT JOIN books AS b ON b.publisher_id = p.publisher_id
WHERE 
b.num_pages > 50
GROUP BY
titel
ORDER BY
n_books DESC
LIMIT 1
'''
wanna_sql(exersice_3)

## Определим автора с самой высокой средней оценкой книг, учитывать будем только книги с 50 и более оценками

In [ ]:
exersice_4 = '''
SELECT a.author AS name,
       ROUND(AVG(rat.rating), 1) as mean_rating
FROM authors AS a
JOIN books AS b ON b.author_id=a.author_id
JOIN ratings AS rat ON rat.book_id=b.book_id
WHERE b.book_id IN (SELECT  book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating) >= 50)
GROUP BY name
ORDER BY mean_rating DESC
LIMIT 1
'''
wanna_sql(exersice_4)

## Посчитаем среднее кол-во обзоров от пользователей, которые поставили больше 48 оценок

In [ ]:
exersice_5 = '''
WITH n_reviews AS (SELECT
                  COUNT(review_id) AS n_reviews
           FROM reviews AS rev
           JOIN
                       (SELECT
                               username,
                               COUNT(rating_id) AS n_ratings
                        FROM
                        ratings AS rat
                        GROUP BY
                        username
                        HAVING 
                        COUNT(rating_id) > 48) AS rat_of_users ON rat_of_users.username = rev.username
                        GROUP BY
                        rev.username)


SELECT
         AVG(n_reviews) AS mean_n_reviews
         FROM n_reviews
'''
wanna_sql(exersice_5)

## Выводы

- Посчитав, сколько книг вышло после 1 января 2000 года - получилось 821 книгa;
- Для каждой книги посчитали количество обзоров и среднюю оценку, выведя их на экран (ТОП3: 1. The Cat in the Hat and Other Dr. Seuss Favorites (кол-во обзоров - 5); 2. A Dirty Job(Grim Reaper) (кол-во обзоров - 5, ср.оценка - 4);  3. School's Out—Forever (Maximum Ride) (кол-во обзоров - 5, ср.оценка - 3)
- Определили издательство, которое выпустило наибольшее число книг толще 50 страниц: им оказалось Penguin Books	с 42 книгами;
- Определили автора с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré cо ср.оценкой 4.3;
- Посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок - получилось 24